Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 папкам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** item_category_id - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


# Задание 1

1. Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).
2. Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).

In [ ]:
#пункт 1
sales_train<-read.csv("sales_train.csv", header = TRUE)
sales_train<-sales_train[c("item_id", "item_cnt_day")]
top_10<-aggregate(sales_train$item_cnt_day, by=list(sales_train$item_id), FUN=sum)
head(top_10[order(top_10$x, decreasing=TRUE),],n=10)


,Group.1,x
,<int>,<dbl>
20603,20949,187642
2750,2808,17245
3655,3732,16642
17419,17717,15830
5718,5822,14515
3657,3734,11688
6544,6675,10289
3654,3731,10099
1815,1855,10032


In [ ]:
#пункт 2
library(dplyr)

sales_train<-read.csv("sales_train.csv", header = TRUE)
sales_train<-sales_train[c("item_id", "date_block_num","item_cnt_day")]
sales_train<-sales_train %>% mutate (y2013=if_else(date_block_num <= 11,1,0))
sales_train<-sales_train %>% mutate (y2014=if_else(date_block_num >= 12 & date_block_num<=23,1,0))
sales_train<-sales_train %>% mutate (y2015=if_else(date_block_num >= 24 & date_block_num<=33,1,0))

df<-aggregate(sales_train[c("item_cnt_day","y2013","y2014","y2015")], by=list(sales_train$item_id), FUN=sum)

df_sort<-df[order(df$item_cnt_day, decreasing=TRUE),]

df_sort<-head(df_sort,n=10)

df_result<-df_sort[df_sort$y2013!=0 & df_sort$y2014!=0 & df_sort$y2015!=0,]
df_result<-df_result[c("Group.1","item_cnt_day")]
colnames(df_result) <- c("item_id","item_cnt_day")
df_result

,item_id,item_cnt_day
,<int>,<dbl>
20603,20949,187642
2750,2808,17245
3655,3732,16642
17419,17717,15830
5718,5822,14515
3657,3734,11688
6544,6675,10289
1815,1855,10032
16494,16787,9227


# Задание 2

> При выполнении на pandas не используйте merge().

1. Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.
2. Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.

In [ ]:
#пункт 1 min
library(dplyr)
items<-read.csv("items.csv", header = TRUE)
item_categories<-read.csv("item_categories.csv", header = TRUE)

table_1<-merge(x=sales_train,y=items,by=c("item_id"))

table_2<-aggregate(table_1$item_cnt_day, by=list(table_1$category_id), FUN=sum)
colnames(table_2) <- c("category_id","item_cnt_day")



min<-min(table_2$item_cnt_day)
table_3<-table_2[table_2$item_cnt_day == min,]

table_4<-merge(x=table_3,y=item_categories,by.x="category_id",by.y="item_category_id")
table_4[c(1,3)]
# table_sort<-table_2[order(table_2$item_cnt_day, decreasing=FALSE),]
# table_sort_min<-head(table_sort,n=2)
# table_min<-merge(x=table_sort_min,y=item_categories, by.x=("category_id"),by.y=("item_category_id"))
# table_min[c(1,3)]





category_id,item_category_name
<int>,<chr>
10,Game consoles - PS2
51,Books - Cognitive literature


In [ ]:
#пункт 2 max
library(dplyr)
items<-read.csv("items.csv", header = TRUE)
item_categories<-read.csv("item_categories.csv", header = TRUE)

table_1<-merge(x=sales_train,y=items,by=c("item_id"))

table_2<-aggregate(table_1$item_cnt_day, by=list(table_1$category_id), FUN=sum)
colnames(table_2) <- c("category_id","item_cnt_day")

max<-max(table_2$item_cnt_day)
table_3<-table_2[table_2$item_cnt_day == max,]

table_4<-merge(x=table_3,y=item_categories,by.x="category_id",by.y="item_category_id")
table_4[c(1,3)]

# table_sort<-table_2[order(table_2$item_cnt_day, decreasing=TRUE),]
# table_sort_max<-head(table_sort,n=1)
# table_max<-merge(x=table_sort_max,y=item_categories, by.x=("category_id"),by.y=("item_category_id"))
# table_max[c(1,3)]

category_id,item_category_name
<int>,<chr>
40,Cinema - DVD


# Задание 3

1. Для категорий из задания 2 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами.
2. Найдите месяцы с минимальными и максимальными продажами для каждого товара из пункта 1.


In [ ]:
#пункт 1, категория 40
library(dplyr)
sales_train<-read.csv("sales_train.csv", header = TRUE)
items<-read.csv("items.csv", header = TRUE)
item_categories<-read.csv("item_categories.csv", header = TRUE)

sales_train_1<-merge(x=sales_train,y=items,all.x=TRUE)
#найдём топ-5 магазинов
shop_top<-aggregate(sales_train$item_cnt_day, by=list(sales_train$shop_id), FUN=sum)
colnames(shop_top) <- c("shop_id","item_cnt_day")

shop_top_5<-head(shop_top[order(shop_top$item_cnt_day,decreasing=TRUE),],n=5)
#оставим только записи из топ-5 магазинов и нужную категорию
shop_table<-filter(sales_train_1,shop_id==31 | shop_id==25 | shop_id==54 | shop_id==28 | shop_id==42)
shop_table_stat<-filter(shop_table,category_id==40)


shop_table_mean<-aggregate(shop_table_stat$item_cnt_day, by=list(shop_table_stat$item_id), FUN=mean)
colnames(shop_table_mean) <- c("item_id","item_cnt_day")
shop_table_mean



item_id,item_cnt_day
<int>,<dbl>
0,1.000000
2,1.000000
3,1.000000
4,1.000000
5,1.000000
6,1.000000
7,1.000000
8,1.000000
9,1.000000


In [ ]:
#пункт 1, категория 10
library(dplyr)
sales_train<-read.csv("sales_train.csv", header = TRUE)
items<-read.csv("items.csv", header = TRUE)
item_categories<-read.csv("item_categories.csv", header = TRUE)

sales_train_1<-merge(x=sales_train,y=items,all.x=TRUE)
#найдём топ-5 магазинов
shop_top<-aggregate(sales_train$item_cnt_day, by=list(sales_train$shop_id), FUN=sum)
colnames(shop_top) <- c("shop_id","item_cnt_day")

shop_top_5<-head(shop_top[order(shop_top$item_cnt_day,decreasing=TRUE),],n=5)
#оставим только записи из топ-5 магазинов и нужную категорию
shop_table<-filter(sales_train_1,shop_id==31 | shop_id==25 | shop_id==54 | shop_id==28 | shop_id==42)
shop_table_stat<-filter(shop_table,category_id==10)

shop_table_stat

# shop_table_mean<-aggregate(shop_table_stat$item_cnt_day, by=list(shop_table_stat$item_id), FUN=mean)
# colnames(shop_table_mean) <- c("item_id","item_cnt_day")
# shop_table_mean


item_id,date,date_block_num,shop_id,item_price,item_cnt_day,item_name,category_id
<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<chr>,<int>


In [ ]:
#пункт 1, категория 51
library(dplyr)
sales_train<-read.csv("sales_train.csv", header = TRUE)
items<-read.csv("items.csv", header = TRUE)
item_categories<-read.csv("item_categories.csv", header = TRUE)

sales_train_1<-merge(x=sales_train,y=items,all.x=TRUE)
#найдём топ-5 магазинов
shop_top<-aggregate(sales_train$item_cnt_day, by=list(sales_train$shop_id), FUN=sum)
colnames(shop_top) <- c("shop_id","item_cnt_day")
shop_top_5<-head(shop_top[order(shop_top$item_cnt_day,decreasing=TRUE),],n=5)

#оставим только записи из топ-5 магазинов и нужную категорию
shop_table<-filter(sales_train_1,shop_id==31 | shop_id==25 | shop_id==54 | shop_id==28 | shop_id==42)
shop_table_stat<-filter(shop_table,category_id==51)
shop_table_stat

# shop_table_mean<-aggregate(shop_table_stat$item_cnt_day, by=list(shop_table_stat$item_id), FUN=mean)
# colnames(shop_table_mean) <- c("item_id","item_cnt_day")
# shop_table_mean


item_id,date,date_block_num,shop_id,item_price,item_cnt_day,item_name,category_id
<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<chr>,<int>


In [ ]:
#пункт 2 min
sales_train_1<-merge(x=sales_train,y=items,all.x=TRUE)
#найдём топ-5 магазинов
shop_top<-aggregate(sales_train$item_cnt_day, by=list(sales_train$shop_id), FUN=sum)
colnames(shop_top) <- c("shop_id","item_cnt_day")
shop_top_5<-head(shop_top[order(shop_top$item_cnt_day,decreasing=TRUE),],n=5)
#оставляем только нужные записи (топ-5 магазинов и категории)
shop_table<-filter(sales_train_1,shop_id==31 | shop_id==25 | shop_id==54 | shop_id==28 | shop_id==42)
shop_table_stat<-filter(shop_table,category_id==51|category_id==10|category_id==40)

shop_table_min<-aggregate(shop_table_stat$date_block_num, by=list(shop_table_stat$item_id), FUN=min)
colnames(shop_table_min) <- c("item_id","date_block_num")
shop_table_min

item_id,date_block_num
<int>,<int>
0,20
2,19
3,18
4,20
5,23
6,18
7,23
8,19
9,19


In [ ]:
#пункт 2 max
sales_train_1<-merge(x=sales_train,y=items,all.x=TRUE)
#найдём топ-5 магазинов
shop_top<-aggregate(sales_train$item_cnt_day, by=list(sales_train$shop_id), FUN=sum)
colnames(shop_top) <- c("shop_id","item_cnt_day")
shop_top_5<-head(shop_top[order(shop_top$item_cnt_day,decreasing=TRUE),],n=5)
#оставляем только нужные записи (топ-5 магазинов и категории)
shop_table<-filter(sales_train_1,shop_id==31 | shop_id==25 | shop_id==54 | shop_id==28 | shop_id==42)
shop_table_stat<-filter(shop_table,category_id==51|category_id==10|category_id==40)

shop_table_max<-aggregate(shop_table_stat$date_block_num, by=list(shop_table_stat$item_id), FUN=max)
colnames(shop_table_max) <- c("item_id","date_block_num")
shop_table_max

item_id,date_block_num
<int>,<int>
0,20
2,22
3,19
4,20
5,23
6,18
7,23
8,20
9,19


In [ ]:
# sales_train %>% group_by(shop_id) %>% tally()
# shop_top<-sales_train %>% group_by(shop_id) %>% summarise(item_cnt_day = n())
# shop_top

#  df_2<-df_1[order(df_1$x, decreasing=TRUE),]

# Задание 4

1. Для категорий из задания 2 найдите среднюю и медианную цену по категории в пределах каждого месяца.
2. В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.

In [ ]:
#пункт 1 категория 40
library(dplyr)
sales_train<-read.csv("sales_train.csv", header = TRUE)
items<-read.csv("items.csv", header = TRUE)
item_categories<-read.csv("item_categories.csv", header = TRUE)

table<-merge(x=sales_train,y=items,all.x=TRUE)
table<-filter(sales_train_1,category_id==40)

table_mean<-aggregate(table$item_price, by=list(table$date_block_num), FUN=mean)
colnames(table_mean) <- c("date_block_num","mean_price")

table_median<-aggregate(table$item_price, by=list(table$date_block_num), FUN=median)
colnames(table_median) <- c("date_block_num","median_price")

table<-merge(x=table_mean, y=table_median,by=c("date_block_num"))
table

date_block_num,mean_price,median_price
<int>,<dbl>,<dbl>
0,244.4859,199.00
1,252.1749,203.00
2,260.6450,231.00
3,254.8524,199.00
4,247.6276,199.00
5,245.8609,199.00
6,236.8121,149.00
7,234.4702,149.00
8,241.8186,198.00


In [ ]:
#пункт 1 категория 10
library(dplyr)
sales_train<-read.csv("sales_train.csv", header = TRUE)
items<-read.csv("items.csv", header = TRUE)
item_categories<-read.csv("item_categories.csv", header = TRUE)

table<-merge(x=sales_train,y=items,all.x=TRUE)
table<-filter(sales_train_1,category_id==10)

table_mean<-aggregate(table$item_price, by=list(table$date_block_num), FUN=mean)
colnames(table_mean) <- c("date_block_num","mean_price")

table_median<-aggregate(table$item_price, by=list(table$date_block_num), FUN=median)
colnames(table_median) <- c("date_block_num","median_price")

table<-merge(x=table_mean, y=table_median,by=c("date_block_num"))
table

date_block_num,mean_price,median_price
<int>,<dbl>,<dbl>
3,1998,1998


In [ ]:
#пункт 1 категория 51
library(dplyr)
sales_train<-read.csv("sales_train.csv", header = TRUE)
items<-read.csv("items.csv", header = TRUE)
item_categories<-read.csv("item_categories.csv", header = TRUE)

table<-merge(x=sales_train,y=items,all.x=TRUE)
table<-filter(sales_train_1,category_id==51)

table_mean<-aggregate(table$item_price, by=list(table$date_block_num), FUN=mean)
colnames(table_mean) <- c("date_block_num","mean_price")

table_median<-aggregate(table$item_price, by=list(table$date_block_num), FUN=median)
colnames(table_median) <- c("date_block_num","median_price")

table<-merge(x=table_mean, y=table_median,by=c("date_block_num"))
table

date_block_num,mean_price,median_price
<int>,<dbl>,<dbl>
18,129,129


In [ ]:
#пункт 2
library(dplyr)
sales_train<-read.csv("sales_train.csv", header = TRUE)
items<-read.csv("items.csv", header = TRUE)

df<-merge(x=sales_train,y=items,all.x=TRUE)
df<-filter(df,category_id==40)

#разделим записи по годам
df<-df %>% mutate (y2013=if_else(date_block_num <= 11,1,0))
df_2013<-df[df$y2013==1,]
df_2013<-df_2013[c("date_block_num","item_id","item_price","category_id")]


df<-df %>% mutate (y2014=if_else(date_block_num >= 12 & date_block_num<=23,1,0))
df_2014<-df[df$y2014==1,]
df_2014<-df_2014[c("date_block_num","item_id","item_price","category_id")]


df<-df %>% mutate (y2015=if_else(date_block_num >= 24 & date_block_num<=33,1,0))
df_2015<-df[df$y2015==1,]
df_2015<-df_2015[c("date_block_num","item_id","item_price","category_id")]





In [ ]:
#рассмотрим 2013 год

df_2013_1<-aggregate(df_2013$item_price, by=list(df_2013$date_block_num), FUN=mean)
colnames(df_2013_1) <- c("date_block_num","item_price_mean")

for(i in 2:length(df_2013_1$item_price_mean)){df_2013_1$test[i]=ifelse(df_2013_1$item_price_mean[i] < df_2013_1$item_price_mean[i-1] & df_2013_1$item_price_mean[i] < df_2013_1$item_price_mean[i+1], print(df_2013_1$date_block_num[i]),NaN)}



[1] 7


In [ ]:
#рассмотрим 2014 год

df_2014_1<-aggregate(df_2014$item_price, by=list(df_2014$date_block_num), FUN=mean)
colnames(df_2014_1) <- c("date_block_num","item_price_mean")

for(i in 2:length(df_2014_1$item_price_mean)){df_2014_1$test[i]=ifelse(df_2014_1$item_price_mean[i] < df_2014_1$item_price_mean[i-1] & df_2014_1$item_price_mean[i] < df_2014_1$item_price_mean[i+1], print(df_2014_1$date_block_num[i]),NaN)}

[1] 16
[1] 21


In [ ]:
#рассмотрим 2015 год

df_2015_1<-aggregate(df_2015$item_price, by=list(df_2015$date_block_num), FUN=mean)
colnames(df_2015_1) <- c("date_block_num","item_price_mean")

for(i in 2:length(df_2015_1$item_price_mean)){df_2015_1$test[i]=ifelse(df_2015_1$item_price_mean[i] < df_2015_1$item_price_mean[i-1] & df_2015_1$item_price_mean[i] < df_2015_1$item_price_mean[i+1], print(df_2015_1$date_block_num[i]),NaN)}

[1] 25
[1] 31
